In [1]:
import numpy as np
import pandas as pd

In [2]:
test_set = pd.read_csv("testing_set_seed123_ml100k.csv",sep=",",
    header=None,
    names=["userID", "itemID", "Rating"],)
test_set

,userID,itemID,Rating
0,0,464,4
1,0,431,2
2,0,179,3
3,0,291,3
4,0,928,4
...,...,...,...
20237,942,450,4
20238,942,858,5
20239,942,157,4
20240,942,24,5


In [3]:
all_users_tp = test_set.groupby('userID')['itemID'].apply(list).tolist()
all_users_tp.__len__()

943

In [4]:
test_set[test_set["userID"]==1]

,userID,itemID,Rating
8,1,746,2
9,1,442,4
10,1,445,2
11,1,280,3
12,1,349,2
13,1,875,4
14,1,1106,4
15,1,370,1
16,1,425,4
17,1,450,4


In [5]:
reco_matrix = np.load("../test/reco_matrix.npy")

In [6]:
reco_matrix[0][1]

array([110, 319, 220, 374, 918, 745,  99, 607,  57, 191])

In [7]:
from HitRation import HitRatio
hr=HitRatio(10)
alist=[]
sum_hr=0
for u in range(reco_matrix[0].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix[0][u])
    # alist.append(x)
    sum_hr=sum_hr+x
    # print(x)
sum_hr/reco_matrix[0].shape[0]
    

0.35949098621421

In [20]:
pd_l29b69= pd.read_csv("../results/res_top10_l29b69.csv",sep=",",)
    # header=None,
    # names=["userID", "itemID", "Rating"],)
reco_matrix_l29b69=[]
for i in range(len(pd_l29b69)):
    reco_matrix_l29b69.append(np.array(pd_l29b69.iloc[i][1:]))
    
# reco_matrix_l49b49

In [22]:
from HitRation import HitRatio
hr=HitRatio(10)
alist=[]
sum_hr=0
for u in range(reco_matrix[0].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix_l29b69[u])
    # alist.append(x)
    sum_hr=sum_hr+x
    # print(x)
sum_hr/reco_matrix[0].shape[0]
    

0.2608695652173913

In [ ]:
pd_l49b49= pd.read_csv("../results/res_top10l49b49.csv",sep=",",)
    # header=None,
    # names=["userID", "itemID", "Rating"],)
reco_matrix_l49b49=[]
for i in range(len(pd_l49b49)):
    reco_matrix_l49b49.append(np.array(pd_l49b49.iloc[i][1:]))
from HitRation import HitRatio
hr=HitRatio(10)
alist=[]
sum_hr=0
for u in range(reco_matrix[0].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix_l49b49[u])
    # alist.append(x)
    sum_hr=sum_hr+x
    # print(x)
sum_hr/reco_matrix[0].shape[0]
    
    
# reco_matrix_l49b49

0.2704135737009544

In [25]:
pd_l69b29= pd.read_csv("../results/res_top10_l69b29.csv",sep=",",)
    # header=None,
    # names=["userID", "itemID", "Rating"],)
reco_matrix_l69b29=[]
for i in range(len(pd_l69b29)):
    reco_matrix_l69b29.append(np.array(pd_l69b29.iloc[i][1:]))
from HitRation import HitRatio
hr=HitRatio(10)
alist=[]
sum_hr=0
for u in range(reco_matrix[0].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix_l69b29[u])
    # alist.append(x)
    sum_hr=sum_hr+x
    # print(x)
sum_hr/reco_matrix[0].shape[0]
    
    
# reco_matrix_l49b49

0.30858960763520676

In [31]:

rating_data_pd = pd.read_csv(
    "../data/ml-100k/indexed_interactions.csv",
    sep="\t",
    header=None,
    names=["userID", "itemID", "Rating", "Timestamp"],
)
rating_data = rating_data_pd.to_numpy()
rating_data.__len__()
rating_data_pd
df_m = pd.read_csv(
    "../data/ml-100K/u.item",
    sep="|",
    names=[
        "movieID",
        "Name",
        "Date",
        "Video_Date",
        "IMDB_URL",
        "unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ],
    header=None,
    encoding="latin-1",
)
print(df_m.shape)
df_m = df_m[
    [
        "movieID",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western",
    ]
]

df_movies_mapped = pd.read_csv(
    "../data/ml-100K/i_id_mapping.csv",
    sep="\t",
    names=["movieID", "itemID"],
    header=None,
    encoding="latin-1",
)
movies = pd.merge(df_m, df_movies_mapped, how="inner", on="movieID")
movies
movies = movies.drop(columns=["movieID"])
movies = movies.sort_values(by="itemID")

unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
genre = movies[unique_genres]
item_features_numpy = genre.to_numpy()

users = pd.read_csv("../data/ml-100k/u_id_mapping.csv", sep="\t")

users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
user_features_numpy = users.to_numpy()
users
def create_genre_column(r):
    all_genres = [g for g in unique_genres if r[g] == 1]
    return "|".join(all_genres)


movies["genres"] = movies.apply(create_genre_column, axis=1)
movies

(1682, 24)


,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,itemID,genres
240,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Comedy
300,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,Thriller|Mystery|Film-Noir|Crime
375,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Children's|Comedy
50,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,3,Romance|Western|War|Drama
344,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,Crime|Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1344,Drama
1192,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1345,Comedy
1176,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1346,Drama
1261,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1347,Drama


In [38]:
import os
os.chdir("/Users/tahsinalamgirkheya/Desktop/work/reranking_fairnes")


In [39]:
from mymetrics.GenrePrecision import GenrePrecision
####Import Metrics####
# from mymetrics.GenrePrecision import GenrePrecision

top_k = 10
###intialize them
gp = GenrePrecision(users, unique_genres, top_k)

/Users/tahsinalamgirkheya/anaconda3/envs/bbmrec/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
gp_1 = gp.compute(np.array(reco_matrix_l29b69),movies)

In [44]:
sum(abs(gp_1))

0.03009362629263124

In [45]:
gp_2 = gp.compute(np.array(reco_matrix_l49b49),movies)
sum(abs(gp_2))

0.02984387180904594

In [46]:
gp_3 = gp.compute(np.array(reco_matrix_l69b29),movies)
sum(abs(gp_3))

0.05702796356030189